#Scraping et import des données de mix énergétique de l'AIE
Module d'import des données des mixes énergétiques de production de l'énergie électrique mis à disposition par l'Agence Internationale de l'Energie.
https://www.iea.org/countries

##Initialisation

###Installations

In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install psycopg2

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,244 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,152 kB]
Get:13 http://security.u

Ces commandes sont utilisées pour télécharger et installer Google Chrome sur un système Linux

In [2]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb

--2023-11-25 13:19:32--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.69.190, 173.194.69.91, 173.194.69.136, ...
Connecting to dl.google.com (dl.google.com)|173.194.69.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103884100 (99M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>]  99.07M   359MB/s    in 0.3s    

2023-11-25 13:19:32 (359 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [103884100/103884100]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following packages were automatically installed and are no longer required:
  apparmor libfuse3-3 liblzo2-2 snapd squashfs-tools
Use 'apt autoremove' to remove them.
The following additional packa

###Import des librairies



*   webdrive utilisée pour automatiser les interactions avec les navigateurs web




In [3]:
import os
import pandas as pd
import psycopg2  # pour se connecter à une base de données PostgreSQL
import requests  # pour effectuer des requêtes web comme GET, POST, etc., pour récupérer des données à partir d'URLs, interagir avec des API, etc

from base64 import b64decode

from selenium import webdriver
#from selenium.webdriver import Chrome, ChromeOptions
#from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.support.ui import WebDriverWait

from time import sleep

ModuleNotFoundError: ignored

###Fonctions utilitaires

La fonction __chrome_driver_setup__ prend en charge la configuration de diverses options pour contrôler le comportement du navigateur lorsqu'il est automatisé par Selenium

In [ ]:
def chrome_driver_setup():
    options = webdriver.ChromeOptions() # pour stocker les options de configuration du navigateur Chrome.
    options.add_argument('--headless')  # pour exécuter le navigateur sans interface graphique
    options.add_argument('--no-sandbox') # pour éviter des erreurs liées à la sécurité
    options.add_argument('--disable-dev-shm-usage') # résoudre certains problèmes de ressources limitées
    options.add_argument("lang=en") # Définit la langue du navigateur sur l'anglais
    options.add_argument("start-maximized") # ouvrir le navigateur en plein écran
    options.add_argument("disable-infobars") #  Désactive les notifications d'informations ou de confirmation
    options.add_argument("--disable-extensions") # Désactive les extensions du navigateur
    options.add_argument("--incognito") # les données de navigation ne seront pas enregistrées
    options.add_argument("--disable-blink-features=AutomationControlled")  # éviter la détection de Selenium

    driver = webdriver.Chrome(options=options) # crée une instance du navigateur Chrome en utilisant les options configurées

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")  # exécute du JavaScript pour remplacer la propriété navigator.webdriver par undefined,
                                                                                                     # ce qui peut aider à contourner la détection de WebDriver par certains sites web

    return driver

La fonction __js_wait_for_text_and_click__ est une fonction en JavaScript qui attend qu'un élément sur une page web corresponde à un sélecteur CSS spécifié et contienne un texte spécifique. Il utilise une boucle d'attente avec un intervalle de 100 millisecondes. Une fois que l'élément correspondant au sélecteur CSS et contenant le texte recherché est trouvé, il arrête la boucle et effectue un clic sur cet élément.

In [ ]:
def js_wait_for_text_and_click(cssSelector, text):
  return "const interval = setInterval(() => {" \
    "	const element = document.querySelector('" + cssSelector + "');" \
    " if (element && element.innerHTML.trim() === '" + text + "') {" \
		"   clearInterval(interval);" \
		"   element.click();" \
    "	}" \
    "}, 100);"

##Scraping des données

Ce code utilise Selenium pour naviguer sur la page web de l'Agence internationale de l'énergie (IEA) pour des pays spécifiés, localiser et récupérer des boutons et des titres spécifiques, puis extraire les données du graphique généré sur cette page.
Voici une explication commentée:

In [25]:
# Fonction pour extraire des données d'un pays sur le site de l'IEA
def scrape_country(driver, country):
  print(f"Accès à la page de l'IEA pour {country}...")                      # Affichage du message indiquant l'accès à la page de l'IEA pour le pays spécifié
  driver.get(f"https://www.iea.org/countries/{country}")                    # Accès à la page web du pays spécifié
  print('   Récupération du bouton de catégorie...')                        # Affichage du message pour récupérer le bouton de catégorie
  sleep(3)                                                                  # Pause de 3 secondes pour permettre le chargement de la page

  # Recherche et récupération du bouton de catégorie
  btn_category = driver.find_element(By.CSS_SELECTOR, '.a-dropdown--indicatorCategory > .a-dropdown__options > div > ul > li:nth-child(9) > button')

  # Vérification si le bouton de catégorie correspond à 'Electricity and heat'
  if btn_category.get_attribute('innerHTML').strip() != 'Electricity and heat':
    print(f"Erreur : impossible de récupérer le bouton de catégorie pour {country}")
    return False

  # Clic sur le bouton de catégorie
  print('   Clic sur le bouton de catégorie...')
  driver.execute_script(js_wait_for_text_and_click('.a-dropdown--indicatorCategory > .a-dropdown__options > div > ul > li:nth-child(9) > button', 'Electricity and heat'))

  # Pause de 15 secondes pour permettre le chargement des données
  sleep(15)

  # Recherche et récupération du bouton d'indicateur
  print('   Récupération du bouton d\'indicateur...')
  btn_indicator = driver.find_element(By.CSS_SELECTOR, '.a-dropdown--selectedIndicator > .a-dropdown__options > div > ul > li:nth-child(6) > button')

  # Vérification si le bouton d'indicateur correspond à 'Electricity generation by source'
  if btn_indicator.get_attribute('innerHTML').strip() != 'Electricity generation by source':
    print(f"Erreur : impossible de récupérer le bouton d'indicateur pour {country}")
    return False

  # Clic sur le bouton d'indicateur
  print('   Clic sur le bouton d\'indicateur...')
  driver.execute_script(js_wait_for_text_and_click('.a-dropdown--selectedIndicator > .a-dropdown__options > div > ul > li:nth-child(6) > button', 'Electricity generation by source'))

  # Pause de 6 secondes pour permettre le chargement des données
  sleep(6)

  # Récupération du titre du graphique
  print('   Récupération du titre du graphique...')
  chart_title = driver.find_element(By.CSS_SELECTOR, 'h3.f-ui-2-bold')

  # Vérification si le titre correspond à 'Electricity generation by source'
  if chart_title.get_attribute('innerHTML').strip()[:32] != 'Electricity generation by source':
    print(f"Erreur : impossible de récupérer le titre du graphique pour {country}")
    return False

  # Attente de la génération du graphique
  print('   Attente de la génération du graphique...')
  sleep(6)

  # Extraction des données du graphique via l'exécution d'un script JavaScript
  print("   Récupération des données du graphique")
  data = driver.execute_script("""const chart = Highcharts.charts[Highcharts.charts.length-1];
    let result = [];
    for(let i = 0; i < chart.series.length; i++) {
      const row = {
            name: chart.series[i].name,
            dataX: chart.series[i].processedXData,
            dataY: chart.series[i].processedYData
        };
      result.push(row);
    }
    return result;""")

  # Retourne les données extraites du graphique
  return data


In [39]:
# Initialisation du pilote Chrome pour l'automatisation du navigateur
driver = chrome_driver_setup()

# Dictionnaire associant les noms de pays à leurs codes respectifs
countries = {
    'austria': 'AT', 'belgium': 'BE', 'czech-republic': 'CZ', 'germany': 'DE', 'spain': 'ES',
    'france': 'FR', 'hungary': 'HU', 'italy': 'IT', 'the-netherlands': 'NL', 'poland': 'PL', 'sweden': 'SE'
}

# Liste pour stocker les données sur les sources d'énergie de différents pays
energy_sources = []

# Parcours des pays et de leurs codes dans le dictionnaire 'countries'
for country, country_code in countries.items():
    # Appel de la fonction scrape_country pour récupérer les données sur les sources d'énergie du pays actuel
    data = scrape_country(driver, country)

    # Vérification si des données ont été récupérées avec succès
    if data:
        # Ajout des données dans la liste 'energy_sources' sous forme de dictionnaire {code_pays: données}
        energy_sources.append({country_code: data})

# Fermeture du pilote Chrome après la collecte des données pour tous les pays
driver.quit()


Accès à la page de l'IEA pour austria...
   Récupération du bouton de catégorie...
   Clic sur le bouton de catégorie...
   Récupération du bouton d'indicateur...
   Clic sur le bouton d'indicateur...
   Récupération du titre du graphique...
   Attente de la génération du graphique...
   Récupération des données du graphique
Accès à la page de l'IEA pour belgium...
   Récupération du bouton de catégorie...
   Clic sur le bouton de catégorie...
   Récupération du bouton d'indicateur...
   Clic sur le bouton d'indicateur...
   Récupération du titre du graphique...
   Attente de la génération du graphique...
   Récupération des données du graphique
Accès à la page de l'IEA pour czech-republic...
   Récupération du bouton de catégorie...
   Clic sur le bouton de catégorie...
   Récupération du bouton d'indicateur...
   Clic sur le bouton d'indicateur...
   Récupération du titre du graphique...
   Attente de la génération du graphique...
   Récupération des données du graphique
Accès à la p

###Augmentation des données
Ajout d'une colonne "CO2e" avec les émissions de GES associées à la source d'énergie primaire

In [27]:
# Création du dictionnaire associant source d'énergie et émissions de GES
co2e_dict = {
    'Natural gas':443,
    'Coal':1058,
    'Biofuels':32,
    'Hydro':10,
    'Oil':778,
    'Nuclear':6,
    'Wind':9.864583,
    'Solar PV':43.9,
    'Geothermal':38,
    'Waste':494,
    'Solar thermal':22,
    'Tide':22,
    'Other sources':411.74
}

###Création du dataframe résultant du scraping

Ce code itère à travers une structure de données energy_sources, probablement une liste ou un ensemble de données relatives à différentes sources d'énergie pour plusieurs pays. Il extrait ces données pour créer un DataFrame pandas appelé df_energy_sources, en ajoutant les informations sur le pays, le type d'énergie, la puissance globale associée, les émissions de CO2 correspondantes et la date des données. Chaque itération dans les boucles imbriquées crée une nouvelle ligne de données dans le DataFrame. Enfin, le DataFrame final est affiché pour montrer les données collectées et structurées.

In [28]:
# Création d'un DataFrame vide avec des colonnes spécifiques
df_energy_sources = pd.DataFrame(columns=['Country', 'Energy', 'PowerGlobal', 'CO2e', 'Date'])

# Parcours des données des sources d'énergie
for data_country in energy_sources:
  # Itération à travers chaque élément de la liste energy_sources
  for country, data1 in data_country.items():
    # Itération à travers les éléments de data_country (pays et données associées)
    for data2 in data1:
      # Itération à travers les données spécifiques (énergie, puissance, etc.)
      for i in range(len(data2['dataX'])):
        # Création d'un dictionnaire représentant une ligne de données
        row = {
            'Country': [country],            # Nom du pays
            'Energy': [data2['name']],     # Type d'énergie
            'PowerGlobal': [data2['dataY'][i]],  # Puissance globale pour cette énergie
            'CO2e': [co2e_dict[data2['name']]],  # Émission de CO2 associée à cette énergie (selon le dictionnaire co2e_dict)
            'Date': [str(data2['dataX'][i]) + '0101']  # Date correspondante à ces données (formaté pour l'année)
        }
        # Ajout de la ligne de données au DataFrame df_energy_sources
        df_energy_sources = pd.concat([df_energy_sources, pd.DataFrame(row)], ignore_index=True)

# Affichage du DataFrame avec les données extraites
display(df_energy_sources)


,Country,Energy,PowerGlobal,CO2e,Date
0,AT,Coal,7006,1058,19900101
1,AT,Coal,7786,1058,19910101
2,AT,Coal,5091,1058,19920101
3,AT,Coal,3740,1058,19930101
4,AT,Coal,3908,1058,19940101
...,...,...,...,...,...
575,BE,Other sources,551,411.74,20170101
576,BE,Other sources,487,411.74,20180101
577,BE,Other sources,436,411.74,20190101
578,BE,Other sources,498,411.74,20200101


###Augmentation des données
Ajout d'une colonne "PowerRelative" représentant la part relative de la source d'énergie primaire dans le mix de production électrique.

In [29]:
# Calcul des totaux de puissance globale pour chaque pays et chaque date
energy_sources_power_totals = df_energy_sources.groupby(['Country','Date'])['PowerGlobal'].sum()
# Fusion des données avec les totaux de puissance globale
df_energy_sources_final = df_energy_sources.merge(energy_sources_power_totals, left_on=['Country', 'Date'], right_on=['Country', 'Date'], how='left')
# Calcul du pourcentage relatif de la puissance globale par rapport au total pour chaque pays et chaque date
df_energy_sources_final.PowerGlobal_y = df_energy_sources_final.PowerGlobal_x / df_energy_sources_final.PowerGlobal_y * 100
# Renommage des colonnes pour plus de clarté
df_energy_sources_final = df_energy_sources_final.rename(columns={'PowerGlobal_x':'PowerGlobal','PowerGlobal_y':'PowerRelative'})
# Sélection des colonnes souhaitées dans le DataFrame final
df_energy_sources_final = df_energy_sources_final[['Country', 'Energy', 'PowerGlobal', 'PowerRelative', 'CO2e', 'Date']]
# Affichage du DataFrame final
display(df_energy_sources_final)


,Country,Energy,PowerGlobal,PowerRelative,CO2e,Date
0,AT,Coal,7006,13.930091,1058,19900101
1,AT,Coal,7786,15.123439,1058,19910101
2,AT,Coal,5091,9.945302,1058,19920101
3,AT,Coal,3740,7.134545,1058,19930101
4,AT,Coal,3908,7.355266,1058,19940101
...,...,...,...,...,...,...
575,BE,Other sources,551,0.636141,411.74,20170101
576,BE,Other sources,487,0.649013,411.74,20180101
577,BE,Other sources,436,0.465608,411.74,20190101
578,BE,Other sources,498,0.557109,411.74,20200101


###Sauvegarde des données en CSV

In [30]:
df_energy_sources_final.to_csv('./energy_sources.csv', index=False)

##Import dans la base de données

###Récupération des données depuis le CSV

In [31]:
df_energy_sources_final = pd.read_csv('./energy_sources.csv')

###Remplacement des valeurs nulles par des 0

In [32]:
df_energy_sources_final.fillna(0, inplace=True)
display(df_energy_sources_final)

,Country,Energy,PowerGlobal,PowerRelative,CO2e,Date
0,AT,Coal,7006.0,13.930091,1058.00,19900101
1,AT,Coal,7786.0,15.123439,1058.00,19910101
2,AT,Coal,5091.0,9.945302,1058.00,19920101
3,AT,Coal,3740.0,7.134545,1058.00,19930101
4,AT,Coal,3908.0,7.355266,1058.00,19940101
...,...,...,...,...,...,...
575,BE,Other sources,551.0,0.636141,411.74,20170101
576,BE,Other sources,487.0,0.649013,411.74,20180101
577,BE,Other sources,436.0,0.465608,411.74,20190101
578,BE,Other sources,498.0,0.557109,411.74,20200101


###Récupération de l'adresse IP du notebook
Permet de whitelister l'adresse pour autoriser l'accès à la base de données.

In [33]:
try:
  response = requests.get('https://api.ipify.org/?format=json', timeout=10)
  print("Adresse IP publique :", response.json()['ip'])
except requests.RequestException:
  print("Impossible de récupérer l'adresse IP publique")

Adresse IP publique : 34.86.129.141


###Définition des constantes

In [34]:
PG_HOST = 'rncp-ol.postgres.database.azure.com'
PG_PORT = 5432
PG_DATABASE = 'rncp_prod'
PG_USERNAME = 'm2i'
PG_PASSWORD = 'Uk5DUF9vbDIwMjM='
PG_CONNECT_TIMEOUT = 10
PG_BATCH_MAX_SIZE = 100
PG_TABLE_ENERGY_SOURCES = 'co2_energy_sources'

###Connexion à la base de données

In [35]:
pg_conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    database=PG_DATABASE,
    user=PG_USERNAME,
    password=b64decode(PG_PASSWORD).decode('utf-8'),
    connect_timeout=PG_CONNECT_TIMEOUT
)
print("Connexion à la base de données PostgreSQL établie avec succès")

pg_conn.autocommit = True

pg_cursor = pg_conn.cursor()

Connexion à la base de données PostgreSQL établie avec succès


###Création de la table si elle n'existe pas

In [36]:
pg_cursor.execute(f"""CREATE TABLE IF NOT EXISTS {PG_TABLE_ENERGY_SOURCES}
  (
    UID INT NOT NULL,
    Country TEXT NOT NULL,
    Energy TEXT NOT NULL,
    PowerGlobal INT NOT NULL,
    PowerRelative FLOAT NOT NULL,
    CO2e FLOAT NOT NULL,
    Date TEXT NOT NULL,
    CONSTRAINT {PG_TABLE_ENERGY_SOURCES}_pk PRIMARY KEY(UID)
  );""")

###Insertion des données dans la base de données
Insertion par lots de PG_BATCH_MAX_SIZE.

In [37]:
# Initialisation d'une liste pour stocker les requêtes SQL d'insertion
queries = []

# Initialisation du compteur de lignes
row_index = 1

# Boucle parcourant chaque ligne du DataFrame df_energy_sources_final
for index, row in df_energy_sources_final.iterrows():
    # Construction d'une chaîne de valeurs représentant une requête SQL d'insertion pour chaque ligne du DataFrame
    queries.append(
        f"'{row['Country']}', '{row['Energy']}', {row['PowerGlobal']}, {row['PowerRelative']}, {row['CO2e']}, '{row['Date']}'")

    # Vérification si le nombre de lignes atteint le seuil de batch défini
    if row_index % PG_BATCH_MAX_SIZE == 0:
        # Affichage du numéro du lot en cours d'insertion dans la base de données
        print(f"Insertion du lot #{(row_index // PG_BATCH_MAX_SIZE)} dans la base de données...")

        # Construction de la requête SQL pour insérer les lignes actuelles dans la base de données
        query = (f"INSERT INTO {PG_TABLE_ENERGY_SOURCES} (Country, Energy, PowerGlobal, PowerRelative, CO2e, Date) "
                 + ", ".join([f"({q})" for q in queries]))  # Utilisation de la méthode join pour combiner les valeurs en une seule chaîne SQL

        # Exécution de la requête SQL avec le curseur PostgreSQL (pg_cursor)
        pg_cursor.execute(query)

        # Réinitialisation de la liste des requêtes pour le prochain lot
        queries = []

    # Incrémentation du compteur de lignes
    row_index += 1

# Vérification s'il reste des requêtes non exécutées
if queries != []:
    # Affichage du numéro du lot pour les requêtes restantes
    print(f"Insertion du lot #{(row_index // PG_BATCH_MAX_SIZE)} dans la base de données...")

    # Construction de la requête SQL pour les requêtes restantes
    query = (f"INSERT INTO {PG_TABLE_ENERGY_SOURCES} (Country, Energy, PowerGlobal, PowerRelative, CO2e, Date) "
             + ", ".join([f"({q})" for q in queries]))

    # Exécution des requêtes restantes avec le curseur PostgreSQL
    pg_cursor.execute(query)

# Message signalant la fin de l'importation des données dans la base de données
print('Importation dans la base de données terminée')


Inserting batch #1 into DB...
Inserting batch #2 into DB...
Inserting batch #3 into DB...
Inserting batch #4 into DB...
Inserting batch #5 into DB...
Inserting batch #5 into DB...
Import into DB completed


###Clôture de la connexion à la base de données

In [38]:
pg_cursor.close()
pg_conn.close()